In [ ]:
##########################################################################
## MISE EN PLACE! ## BLOCK 1
#------------------

## Import math and graphics packages
from datetime import datetime
import matplotlib.pyplot as plt
from metpy.plots import SkewT
from metpy.units import pandas_dataframe_to_unit_arrays, units
import numpy as np
import metpy.calc as mpcalc
from metpy.cbook import get_test_data
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import pandas as pd

In [ ]:
##########################################################################
## READ IN DATA AND ORGANIZE VARIABLES ## BLOCK 2
#------------------

col_names = ['pressure', 'height', 'temperature', 'dewpoint', 'direction', 'speed']

df = pd.read_fwf(get_test_data('nov11_sounding.txt', as_file_obj=False),
                 skiprows=5, usecols=[0, 1, 2, 3, 6, 7], names=col_names)

# Drop any rows with all NaN values for T, Td, winds
df = df.dropna(subset=('temperature', 'dewpoint', 'direction', 'speed'
                       ), how='all').reset_index(drop=True)

# We will pull the data out of the example dataset into individual variables and
# assign units.
p = df['pressure'].values * units.hPa
T = df['temperature'].values * units.degC
Td = df['dewpoint'].values * units.degC
z = (df['height'].values/1000) * units.km
wind_speed = df['speed'].values * units.knots
wind_dir = df['direction'].values * units.degrees
u, v = mpcalc.wind_components(wind_speed, wind_dir)

In [ ]:
##########################################################################
# CHANGE PARAMETERS/SCENARIOS ## BLOCK 3
#------------------

# You can use this block to modify the existing sounding data. For the purposes
# of this practical, I have set up three scenarios below. To use them, be sure 
# to only remove the hashtags in front of the T and Td variables for each 
# scenario before running the code.

# SCENARIO No. 1
T = (df['temperature'].values *0.8) * units.degC
Td = (df['dewpoint'].values *0.8) * units.degC

# SCENARIO No. 2
#T = (df['temperature'].values *1.5) * units.degC
#Td = (df['dewpoint'].values *1.5) * units.degC

# SCENARIO No. 3
#T = (df['temperature'].values - 25) * units.degC
#Td = (df['dewpoint'].values - 20) * units.degC

In [ ]:
##########################################################################
# Pressure vs elevation plot ## BLOCK 4
#------------------

plt.plot(p,z)
#plt.plot(np.log(p.magnitude),z)
plt.grid()
plt.show()

In [ ]:
##########################################################################
# Temperature vs elevation plot ## BLOCK 5
#------------------

plt.plot(T,z)
plt.grid()
plt.show()

In [ ]:
##########################################################################
# Pressure vs temperature plot ## BLOCK 6
#------------------

plt.yscale("log") 
plt.plot(T,p)
plt.gca().invert_yaxis()
plt.grid()
plt.show()

In [ ]:
##########################################################################
# Advanced Skew-T Plotting ## BLOCK 7
# ------------------------
#
# Fiducial lines indicating dry adiabats, moist adiabats, and mixing ratio are
# useful when performing further analysis on the Skew-T diagram. Often the
# 0C isotherm is emphasized and areas of CAPE and CIN are shaded.
#
# * **Lifting Condensation Level (LCL)** - The level at which an air parcel's
#   relative humidity becomes 100% when lifted along a dry adiabatic path.
# * **Parcel Path** - Path followed by a hypothetical parcel of air, beginning
#   at the surface temperature/pressure and rising dry adiabatically until
#   reaching the LCL, then rising moist adiabatially.

# Create a new figure. The dimensions here give a good aspect ratio
fig = plt.figure(figsize=(9, 9))
skew = SkewT(fig, rotation=30)

# Plot the data using normal plotting functions, in this case using
# log scaling in Y, as dictated by the typical meteorological plot
skew.plot(p, T, 'r')
skew.plot(p, Td, 'g')
skew.ax.set_ylim(1000, 40)
skew.ax.set_xlim(-80, 60)

# Calculate the LCL and plot LCL temperature as black dot
lcl_pressure, lcl_temperature = mpcalc.lcl(p[0], T[0], Td[0])
print(lcl_pressure, lcl_temperature)
skew.plot(lcl_pressure, lcl_temperature, 'ko', markerfacecolor='black')

# Calculate the parcel profile.
parcel_prof = mpcalc.parcel_profile(p, T[0], Td[0]).to('degC')

# Plot the parcel profile as a black line
skew.plot(p, parcel_prof, 'k', linewidth=2)

# Calculate values of CAPE and CIN
cape_mag, cin_mag = mpcalc.cape_cin(p, T, Td, parcel_prof, which_lfc='bottom', which_el='top')
print(cape_mag, cin_mag)

# Shade areas of CAPE and CIN
skew.shade_cin(p, T, parcel_prof, Td)
skew.shade_cape(p, T, parcel_prof)

# Plot a zero degree isotherm
skew.ax.axvline(0, color='c', linestyle='--', linewidth=2)

# Add the relevant special lines
skew.plot_dry_adiabats()
skew.plot_moist_adiabats()
skew.plot_mixing_lines()

# Show the plot
plt.show()

In [ ]:
##########################################################################
# Stripped-down Skew-T Plotting ## BLOCK 8
# ------------------------

# Create a new figure. The dimensions here give a good aspect ratio
fig = plt.figure(figsize=(9, 9))
skew = SkewT(fig, rotation=30)

# Plot the data using normal plotting functions, in this case using
# log scaling in Y, as dictated by the typical meteorological plot
skew.plot(p, T, 'r')
skew.plot(p, Td, 'g')
skew.ax.set_ylim(1000, 40)
skew.ax.set_xlim(-80, 60)

# Plot LCL temperature as black dot
skew.plot(lcl_pressure, lcl_temperature, 'ko', markerfacecolor='black')

# Plot the parcel profile as a black line
skew.plot(p, parcel_prof, 'k', linewidth=2)

# Shade areas of CAPE and CIN
skew.shade_cin(p, T, parcel_prof, Td)
skew.shade_cape(p, T, parcel_prof)

# Show the plot
plt.show()